<header style="padding:1px;background:#f9f9f9;border-top:3px solid #00b2b1"><img id="Teradata-logo" src="https://www.teradata.com/Teradata/Images/Rebrand/Teradata_logo-two_color.png" alt="Teradata" width="220" align="right" />

<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>Energy Consumption Forecasting</b>
</header>

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Introduction:</b></p>

<p style = 'font-size:16px;font-family:Arial'>For energy trading companies, forecasting electricity consumption is one key driver in building a successful business. Proper forecasting of market demand prevents losses (in case of overselling energy to market) and lost profits (in case of underestimating demand). Also, the regulator of the energy market can apply fees or even disqualify a trading company for specific periods in case of frequent inaccurate forecasts. This is why increasing the accuracy even by 0.1% can significantly improve the profitability of the energy trading company.</p>

<p style = 'font-size:16px;font-family:Arial'>In this demo, we demonstrate how the entire lifecycle of consumption forecast can be implemented using Vantage technologies and, specifically, the combination of Bring Your Own Model (BYOM), Vantage Analytics Library (VAL) and teradataml python client library solution. This demo consists of four parts (details on Teradata's "Analytics 1-2-3" strategy can be found <a href = 'https://assets.teradata.com/resourceCenter/downloads/WhitePapers/Analytics-123-Enabling-Enterprise-AI-at-Scale-MD006623.pdf'>here</a>):</p>
<ol style = 'font-size:16px;font-family:Arial'>
    <li>Data discovery using client python libraries</li>
    <li>Feature Prep and Transformation using Vantage Analytic Library</li>
    <li>Model training using the scikit-learn</li>
    <li>Scoring the model in Vantage and analyzing the results</li>
    </ol>

<p style = 'font-size:16px;font-family:Arial'>The dataset used in this demo represents electricity consumption in Norway from the 1st of January 2016 to the 31st of August 2019. Each line in this dataset reflects consumption for one hour. Apart from electricity consumption, this datamart also reflects additional data: weather from multiple sources, daylight information and labour calendar. We collected all data from open data sources.</p>

<b style = 'font-size:18px;font-family:Arial;color:#E37C4D'>Utilize Vantage to Operationalize the Machine Learning Process</b>

<p style = 'font-size:16px;font-family:Arial'Open-source tools and techniques provide a rich ecosystem for data scientists and analysts to gain new insights into their data.  However, the process of obtaining these insights is manual, error-prone, and time-consuming process.  Most machine learning tools and platforms seek to make model training more efficient and ignore the more significant challenges with;</p>

<ul style = 'font-size:16px;font-family:Arial'>
    <li>Data Discovery and Statistical Analysis</li>
    <li>Data Preparation and Feature Engineering</li>
    <li>Model Deployment and Evaluation At Operational Scale</li>
    </ul>

<p style = 'font-size:16px;font-family:Arial'>Traditional approaches require the developer to move data <b>from</b> the sources <b>to</b> the analytics.  Even "integrated" analytic systems like Apache Spark provide parallel processing for analyzing data but don't optimize for loading data - neither locality nor quantity that needs to be moved.</p>

<p style = 'font-size:16px;font-family:Arial'>Teradata Vantage reverses this model; and allows PUSH processing down to the individual processing nodes where the data resides.  This allows for the unprecedented scale of the analytical processing, reduced costs in data movement/egress charges, and drastically improved performance.</p> 

<hr>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Downloading and installing additional software needed</b>

In [ ]:
# %%capture
# # '%%capture' suppresses the display of installation steps of the following packages
# !pip install sklearn2pmml
# !pip install jdk4py
# !pip install teradataml

<div class="alert alert-block alert-info">
    <p style = 'font-size:16px;font-family:Arial'><b>Note: </b><i>The above statements may need to be uncommented if you run the notebooks on a platform other than ClearScape Analytics Experience that does not have the libraries installed. If you uncomment those installs, be sure to restart the kernel after executing those lines to bring the installed libraries into memory. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>
</div>

<p style = 'font-size:16px;font-family:Arial'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
import os
import getpass
import sys
import warnings

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd

from jdk4py import JAVA, JAVA_HOME, JAVA_VERSION

from teradataml import *
from teradataml.analytics.valib import *
from teradataml.analytics.Transformations import *
from teradataml.dataframe.copy_to import copy_to_sql
from teradataml.dataframe.dataframe import DataFrame, in_schema
from teradataml.context.context import create_context, remove_context, get_connection
from teradataml.options.display import display

from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler

from sklearn2pmml.pipeline import PMMLPipeline
from sklearn2pmml import sklearn2pmml
import seaborn as sns

# Suppress warnings
warnings.filterwarnings('ignore')
display.max_rows = 5

# Modify the following to match the specific client environment settings
configure.val_install_location = 'val'
configure.byom_install_location = 'mldb'
os.environ['PATH'] = os.pathsep.join([os.environ['PATH'], str(JAVA_HOME), str(JAVA)[:-5]])

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>1. Connect to Vantage</b>
<p style = 'font-size:16px;font-family:Arial'>You will be prompted to provide the password. Enter your password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username = 'demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=Energy_Consumption_Forecasting_Python.ipynb;' UPDATE FOR SESSION;''')

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

<p style = 'font-size:20px;font-family:Arial;color:#E37C4D'><b>Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial'>We have provided data for this demo on cloud storage. You can either run the demo using foreign tables to access the data without any storage on your environment or download the data to local storage, which may yield faster execution. Still, there could be considerations of available storage. Two statements are in the following cell, and one is commented out. You may switch which mode you choose by changing the comment string.</p>

In [ ]:
%run -i ../run_procedure.py "call get_data('DEMO_Energy_cloud');"        # Takes 1 minute
# %run -i ../run_procedure.py "call get_data('DEMO_Energy_local');"        # Takes 2 minutes

<p style = 'font-size:16px;font-family:Arial'>Next is an optional step – if you want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"        # Takes 10 seconds

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>2. Data Exploration</b>

<table style = 'width:100%;table-layout:fixed;'>
<tr>
    <td style = 'vertical-align:middle' width = '50%'>
        <p style = 'font-size:16px;font-family:Arial'>Users can access large volumes of data by connecting remotely using the teradataml client connection library.  Python methods are translated to SQL and run remotely on the Vantage system.  Only the minimal amount of data required is copied to the client, allowing users to interact with data sets of any size and scale.
    </td>
    <td><img src = 'images/connect_and_discover.png' width = '400'></td>
</tr>
</table>

<p style = 'font-size:16px;font-family:Arial'>Create a "Virtual DataFrame" that points to the data set in Vantage</p>

In [ ]:
df = DataFrame(in_schema("DEMO_Energy", "consumption"))
df.shape

<p style = 'font-size:16px;font-family:Arial'>Let's investigate the data by looking at a data sample.</p>

In [ ]:
df

<p style = 'font-size:16px;font-family:Arial'>The dataset above shows the hourly consumption of energy. Multiple columns are potential factors affecting energy consumption, such as is_dark, is_holiday, etc.</p> 

In [ ]:
# Convert to pandas dataframe
pd_df = df.to_pandas(all_rows = True)

# Set the size of the plot
plt.figure(figsize = (12, 6))

# Create line plot using seaborn
sns.set_palette(['#add8e6', '#90ee90', '#00bfff'])
sns.lineplot(data = pd_df, x = 'TD_TIMECODE', y = 'consumption')

# Add x label
plt.xlabel('Date', fontsize = 12)

# Add y label
plt.ylabel('Energy Units', fontsize = 12)

# Add title
plt.title('Energy Demand', fontsize = 16)

# Add legend
plt.legend(labels = ['Energy'], fontsize = 12)

# Add grid lines
plt.grid(axis = 'y', alpha = 0.5)

# Remove spines
sns.despine()

# Show the plot
plt.show()

In [ ]:
# Normalize the data
scaler = MinMaxScaler()
cols = ['cap_air_temperature', 'cap_cloud_area_fraction', 'cap_precipitation_amount']
pd_df[cols] = scaler.fit_transform(pd_df[cols])

# Create three subplots
fig, axs = plt.subplots(nrows = 3, ncols = 1, figsize = (12, 9))
sns.set_palette(['#add8e6', '#90ee90', '#00bfff'])
sns.lineplot(x = 'TD_TIMECODE', y = 'cap_air_temperature', data = pd_df, ax = axs[0])
sns.lineplot(x = 'TD_TIMECODE', y = 'cap_cloud_area_fraction', data = pd_df, ax = axs[1])
sns.lineplot(x = 'TD_TIMECODE', y = 'cap_precipitation_amount', data = pd_df, ax = axs[2])

# Set the labels, titles, and other properties for each subplot
cols = ['cap_air_temperature', 'cap_cloud_area_fraction', 'cap_precipitation_amount']
for i, ax in enumerate(axs):
    ax.set_ylabel('Normalized Values')
    ax.set_title(cols[i])
    ax.set_xlabel('Date')
    ax.grid()
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
    fig.autofmt_xdate(rotation=45)

plt.tight_layout()
plt.show()

<p style = 'font-size:16px;font-family:Arial'>The graph of cap_air_temperature shows an inverse relationship with energy consumption, meaning that in countries with colder climates like Norway, electricity usage tends to increase as the temperature drops, likely due to increased demand for heating. Conversely, electricity usage tends to decrease when the temperature rises, potentially due to reduced need for heating.</p> 

In [ ]:
pd_df['quarter'] = pd_df['TD_TIMECODE'].dt.quarter
# create boxplots for selected columns for each quarter
sns.boxplot(x = 'quarter', y = 'consumption', data = pd_df, palette = 'pastel')

<p style = 'font-size:16px;font-family:Arial'>The above graph shows the distribution of energy consumption across quarters. It indicates that the 1st and 4th quarters across years witness high energy consumption due to cold weather, while the 3rd quarter witnesses the least energy consumption across years, indicating the summer season.</p> 

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>3. Data Preparation</b>

<table style = 'width:100%;table-layout:fixed;'>
<tr>
    <td style = 'vertical-align:top' width = '50%'>
        <p style = 'font-size:16px;font-family:Arial'>The Vantage Analytic Library is a suite of powerful functions that allows for whole-data-set descriptive analysis, data transformation, hypothesis testing, and algorithmic algorithms at an extreme scale.  As with all Vantage capabilities, these functions run in parallel at the source of the data</p>
        <ol style = 'font-size:16px;font-family:Arial'>
            <li>Create Feature Transformation objects</li>
            <br>
            <li>Define the columns to be retained in the analytic data set</li>
            <br>
            <li>Push the transformations to the data in Vantage</li>
            <br>
            <li>Inspect the results</li>
        </ol>
    </td>
    <td><img src = 'images/VAL_transformation.png' width = '400'></td>
</tr>
</table>

In [ ]:
weekday_mapping = {1:'monday', 2:'tuesday', 3:'wednesday', 4:'thursday', 5:'friday', 6:'saturday', 7:'sunday'}
weekday_t = OneHotEncoder(values = weekday_mapping, columns = 'weekday')

hour_t = OneHotEncoder(values = [x for x in range(0,24)],  columns = 'h')

rs = MinMaxScalar(columns = ['nasa_temp','cap_air_temperature', 'cap_cloud_area_fraction', 'cap_precipitation_amount'])

rt = Retain(columns = ['consumption',
                       'is_dark', 'is_light', 'is_from_light_to_dark', 'is_from_dark_to_light', 
                       'is_holiday', 'is_pre_holiday'])

<p style = 'font-size:16px;font-family:Arial'>The transformation objects created in the previous step will be used to prepare the data for modeling. Specifically, weekday_t and hour_t will be used to convert weekday and hour columns from numeric to one-hot encoded columns. rs will be used to scale the nasa_temp using MinMaxScalar, and rt will be used to retain the specified columns. These transformations will enable the data to be used effectively in a machine learning model.</p>

In [ ]:
t_output = valib.Transform(data = df,
                           one_hot_encode = [weekday_t, hour_t], 
                           rescale = [rs], 
                           index_columns = 'TD_TIMECODE',
                           retain = [rt])

In [ ]:
t_output.result

<p style = 'font-size:16px;font-family:Arial'>Please scroll to the right and observe that we now have columns named <b>monday-sunday</b> and <b>0_h - 23_h</b>. Also, nasa_temp has been scaled.</p>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>4. Model Training</b>

<table style = 'width:100%;table-layout:fixed;'>
<tr>
    <td style = 'vertical-align:top' width = '50%'>
        <p style = 'font-size:16px;font-family:Arial'>With Vantage Bring Your Own Model, users can take advantage of a rich ecosystem of Machine Learning, Data Preparation, and Advanced analytical libraries available in the open-source and commercial space.  This demonstration illustrates how to utilize simple client-side training pipelines</p>
        <ol style = 'font-size:16px;font-family:Arial'>
            <li>Create Train and Test data sets in Vantage</li>
            <br>
            <li>Copy the training data to the client</li>
            <br>
            <li>Prepare data and train the model</li>
            <br>
            <li>Load the model into Vantage</li>
        </ol>
    </td>
    <td><img src = 'images/BYOM_model_training.png' width = '400'></td>
</tr>
</table>

In [ ]:
copy_to_sql(t_output.result.iloc[int(t_output.result.shape[0])-168:],
            table_name = 'energy_consumption_variables_rescaled_test',
            if_exists = 'replace')

copy_to_sql(t_output.result.iloc[:int(t_output.result.shape[0])-168],
            table_name = 'energy_consumption_variables_rescaled_train',
            if_exists = 'replace')

<p style = 'font-size:16px;font-family:Arial'>The above step creates training and testing datasets. The last 168 hours, i.e. seven days, are used for testing, and the remaining data is used for training.</p>

In [ ]:
df = pd.read_sql('SELECT * FROM energy_consumption_variables_rescaled_train order by TD_TIMECODE;', eng)

<p style = 'font-size:16px;font-family:Arial'>We calculate the average consumption for the last day of the training period. We will use this number for the normalization of the target variable.</p>

In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)

normalize_value = int(df.tail(24).mean()['consumption'])
normalize_value

In [ ]:
train_x = df.drop(['TD_TIMECODE', 'consumption'], axis = 1).astype(float)
train_y = df['consumption'] - normalize_value

feature_names = list(train_x.columns)

<p style = 'font-size:16px;font-family:Arial'>Dropping TD_TIMECODE and consumption columns from the training dataset is done because TD_TIMECODE is a timestamp and consumption is the target variable we want to predict, so we can't use it as a feature in our model. Normalizing the target variable by subtracting the normalized_value is done to scale the consumption values and bring them in a similar range as the other features used for prediction. This helps the model to learn the correct relationship between features and the target variable, and makes the training process more efficient.</p>

In [ ]:
pipeline_lr = PMMLPipeline(steps = [('lr', LinearRegression())])
pipeline_rf = PMMLPipeline(steps = [('random_forest', RandomForestRegressor(n_estimators = 100, max_depth = 10, random_state = 42))])

pipeline_lr.fit(train_x,train_y)
pipeline_rf.fit(train_x,train_y)

sklearn2pmml(pipeline_lr, "energy_consumption_LR.pmml", with_repr = True)
sklearn2pmml(pipeline_rf, "energy_consumption_RF.pmml", with_repr = True)

<p style = 'font-size:16px;font-family:Arial'>The above step creates two PMML Pipelines, one with a Linear Regression object and another with a Random Forest object. These Pipelines are used to train the models using the "fit" method with the preprocessed training dataset. The trained pipelines are then saved as PMML files locally for future use.</p>

In [ ]:
# Load the PMML file into Vantage

model_ids = ['lr', 'rf']
model_files = ['energy_consumption_LR.pmml', 'energy_consumption_RF.pmml']
table_name = 'energy_models'

for model_id, model_file in zip(model_ids, model_files):
    try:
        save_byom(model_id = model_id, model_file = model_file, table_name = table_name)
    except Exception as e:
        # if our model exists, delete and rewrite
        if str(e.args).find('TDML_2200') >= 1:
            delete_byom(model_id = model_id, table_name = table_name)
            save_byom(model_id = model_id, model_file = model_file, table_name = table_name)
        else:
            raise ValueError(f"Unable to save the model '{model_id}' in '{table_name}' due to the following error: {e}")

# Show the bank_models table
list_byom(table_name)

<p style = 'font-size:16px;font-family:Arial'>In the above steps, the trained PMML models are saved in a table named "energy_models". If a model with the same model_id already exists, it is deleted first and then the latest trained model is saved again using the save_byom method. This ensures that the most recent version of the model is always stored in the table.</p>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>5. Model Scoring and Evaluation</b>

<table style = 'width:100%;table-layout:fixed;'>
<tr>
    <td style = 'vertical-align:top' width = '50%'>
        <p style = 'font-size:16px;font-family:Arial'>The final step in this process is to test the trained model.  The PMMLPredict function will take the stored pipeline object (including any data preparation and mapping tasks) and execute it against the data on the Vantage Nodes.  Note that we can keep many models in the model table, with versioning, last scored timestamp, or any other management data to allow for the operational management of the process.</p>
        <ol style = 'font-size:16px;font-family:Arial'>
            <li>Create a pointer to the model in Vantage</li>
            <br>
            <li>Execute the Scoring function using the model against the testing data</li>
            <br>
            <li>Copy the results of the test to the client - only needs to be a subset of rows if desired</li>
            <br>
            <li>Visualize the results</li>
        </ol>
    </td>
    <td><img src = 'images/Score_and_Evaluate.png' width = '400'></td>
</tr>
</table>

In [ ]:
# Obtain a pointer to the model
table_name = 'energy_models'
model_id = 'lr'
model_lr = retrieve_byom(model_id, table_name = table_name)
df_test = DataFrame('energy_consumption_variables_rescaled_test')

result_lr = PMMLPredict(
            modeldata = model_lr,
            newdata = df_test,
            accumulate = ['TD_TIMECODE','consumption'],
            ).result.to_pandas(all_rows = True)

In [ ]:
result_lr

<p style = 'font-size:16px;font-family:Arial'>In the above step, we use the PMMLPredict method from teradataml library to score the model in the database. The PMMLPredict function in Teradata ML-Engine allows users to score the PMML model directly on the data in the Vantage system, without having to move the data or the model outside the system. This can help to improve the efficiency and security of the scoring process.</p>

In [ ]:
# Obtain a pointer to the model
table_name = 'energy_models'
model_id = 'rf'
model_rf = retrieve_byom(model_id, table_name = table_name)
df_test = DataFrame('energy_consumption_variables_rescaled_test')

result_rf = PMMLPredict(
            modeldata = model_rf,
            newdata = df_test,
            accumulate = ['TD_TIMECODE','consumption'],
            ).result.to_pandas(all_rows = True)

In [ ]:
result_rf

<p style = 'font-size:16px;font-family:Arial'>In the above step, we use the PMMLPredict method from teradataml library to score the model in the database. The PMMLPredict function in Teradata ML-Engine allows users to score the PMML model directly on the data in the Vantage system, without having to move the data or the model outside the system. This can help to improve the efficiency and security of the scoring process.</p>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>6. Visualize the results</b>

In [ ]:
# Read the data and preprocess
df_lr = result_lr
df_rf = result_rf

# Calculate RMS errors
rms_lr = mean_squared_error(result_lr['consumption'], result_lr['prediction'].astype(float) + normalize_value, squared = False)
rms_rf = mean_squared_error(result_rf['consumption'], result_rf['prediction'].astype(float) + normalize_value, squared = False)

# Create the plot
fig, ax = plt.subplots(figsize=(14, 6))

# Add the first plot (linear regression)
ax.plot(df_lr.index, df_lr['consumption'], label = f'Actual Consumption', color = 'red', linewidth = 2)
ax.plot(df_lr.index, df_lr['prediction'].astype(float) + normalize_value, label = f'Linear Regression (RMS = {rms_lr:.2f})', color = 'blue', linestyle = '--')

# Add the second plot (random forest)
ax.plot(df_rf.index, df_rf['prediction'].astype(float) + normalize_value, label=f'Random Forest (RMS = {rms_rf:.2f})', color = 'green', linestyle = '--')

# Set the axis labels, title, and legend
ax.set_xlabel('Date')
ax.set_ylabel('Energy Consumption')
ax.set_title('Energy Consumption Prediction')
ax.legend()

# Add gridlines
ax.grid(axis = 'y', linestyle = '--')

# Add a background color
fig.patch.set_facecolor('#f2f2f2')

# Display the plot
plt.show()

<p style = 'font-size:16px;font-family:Arial'>The above graph displays the Root Mean Squared (RMS) error values for both Linear Regression and Random Forest models. The lower the RMS error value, the better the model's performance. As we can see, Random Forest outperforms Linear Regression in predicting energy demand, as it has a lower RMS error value. Therefore, Random Forest is more suitable for proactively predicting energy demand in our use case.</p>

<p style = 'font-size:16px;font-family:Arial'>This demonstration has illustrated a simplified - but complete - overview of how a typical machine learning workflow can be improved using Vantage in conjunction with open-source tools and techniques.  This combination allows users to leverage open-source innovation with Vantage's operational scale, power, and stability.</p>

<hr>
<b style = 'font-size:28px;font-family:Arial;color:#E37C4D'>7. Cleanup</b>
<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>Cleanup work tables to prevent errors next time.</p>

In [ ]:
tables = ['energy_models', 'energy_consumption_variables_rescaled_train', 'energy_consumption_variables_rescaled_test']

# Loop through the list of tables and execute the drop table command for each table
for table in tables:
    try:
        db_drop_table(table_name = table)
    except:
        pass

<p style = 'font-size:18px;font-family:Arial;color:#E37C4D'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_Energy');"        # Takes 5 seconds

In [ ]:
remove_context()

<footer style="padding:10px;background:#f9f9f9;border-bottom:3px solid #394851">©2023 Teradata. All Rights Reserved</footer>